
## 因子评测——货币资金/市值

这个因子缘于自己炒股时的发现，某些股票会出现市值低于账面货币资金的情况，拍脑门地想，这肯定是低估的一种表现。

虽然企业基本都是负债经营，但如果长期经营后账面能有很多现金，那肯定是其业务不需要消耗现金也能继续运转，所以这可能是一个好的指标。

这个指标还不同于常说的实现率（市值/现金流净额），现金流净额是动态的，临时性的，而这个指标是考验长期积累的成过，即质量。

先来看看目前市场上 <b>*现金/市值比*</b> 最大的30股票都是哪些。

In [9]:
def get_top_cashr(date = None,k = 30, tra_ass = True, ex_fin = False):
    if(date == None):
        date = datetime.datetime.now()
        # maybe should be the current day
    
    if not ex_fin:
        q = query(valuation.code, valuation.market_cap, balance.cash_equivalents, balance.trading_assets).limit(10000)
    else:
        # 剔除银行，非银金融和房地产
        banks = get_industry_stocks('801780',date = date) + get_industry_stocks('801180',date = date) + get_industry_stocks('801790',date=date)
        q = query(valuation.code, valuation.market_cap, balance.cash_equivalents, balance.trading_assets
                 ).filter(valuation.code.notin_(banks)).limit(10000)
   
    df = get_fundamentals(q, date = date)
    # 是否包含交易性金融资产
    if(tra_ass == True):
        df['cashr'] = (df['cash_equivalents'] + df['trading_assets'])/1e8 / df['market_cap']
    else:
        df['cashr'] = df['cash_equivalents'] / 1e8 / df['market_cap']
    
    df = df.sort_values(by = 'cashr', ascending = False)[:k]
    
    df2 = get_all_securities(date = date)
    df2 = df2['display_name']
    df = df.join(df2, on = 'code')
    return(df)

In [10]:
# 包含交易性金融资产，不剔除特别行业
df = get_top_cashr()
df

,code,market_cap,cash_equivalents,trading_assets,cashr,display_name
2960,000046.XSHE,109.6398,1.907257e+10,3.167195e+10,4.628294,泛海控股
4127,601229.XSHG,1064.0690,1.629686e+11,2.876929e+11,4.235266,上海银行
4105,601169.XSHG,962.0058,1.814780e+11,2.230120e+11,4.204652,北京银行
4115,600919.XSHG,1008.7653,1.523082e+11,2.609285e+11,4.096460,江苏银行
3815,601577.XSHG,342.2342,4.821481e+10,9.110414e+10,4.070866,长沙银行
4246,601658.XSHG,4646.9136,1.253073e+12,5.286900e+11,3.834293,邮储银行
4094,600015.XSHG,889.3815,1.874410e+11,1.367870e+11,3.645545,华夏银行
4213,601998.XSHG,2461.4226,4.266040e+11,4.645750e+11,3.620585,中信银行
3803,600705.XSHG,336.2830,7.931880e+10,4.147962e+10,3.592166,中航产融
2620,600466.XSHG,80.4257,2.681097e+10,1.486152e+07,3.335480,蓝光发展


看到这个结果我还是挺震惊的，虽然第30名的指标高达2.34

但目前排名第一的<b>泛海控股</b>已经暴雷，股价跌得惨不忍睹，列表中还有<b>蓝光发展</b>也是近期暴雷的

除此之外，前30的榜单几乎被银行、证券和房地产行业垄断

从这里可以看出，金融业和房地产业（也可以算金融业了）手上的货币资金最多，但<b>远远不能说明其安全</b>

这与我起初设想货币资金是企业经营积累所得非常不同，金融业的高额现金储备几乎都靠发债而来

在危机来临时，这些看似很多的现金可能远远不能覆盖其债务

下面我们剔除金融和房地产行业试试

In [11]:
# 包含交易性金融资产，剔除特定行业
df = get_top_cashr(ex_fin=True)
df

,code,market_cap,cash_equivalents,trading_assets,cashr,display_name
2420,600170.XSHG,241.3092,6.456994e+10,2.240229e+09,2.768654,上海建工
2408,600153.XSHG,217.6300,5.840044e+10,1.182677e+09,2.737817,建发股份
2657,600502.XSHG,65.9204,1.351372e+10,7.000000e+07,2.060624,安徽建工
2383,600122.XSHG,17.9530,6.312607e+08,2.735949e+09,1.875569,ST宏图
2921,600853.XSHG,24.1176,3.907026e+09,8.402000e+07,1.654827,龙建股份
92,000521.XSHE,34.6806,4.408281e+09,1.028681e+09,1.567724,长虹美菱
3049,601186.XSHG,989.9486,1.375106e+11,2.579028e+09,1.415120,中国铁建
2791,600686.XSHG,49.9065,4.060115e+09,2.983672e+09,1.411397,金龙汽车
2989,600981.XSHG,66.8245,4.393715e+09,4.716495e+09,1.363304,汇鸿集团
2912,600839.XSHG,130.6397,1.632814e+10,1.406335e+09,1.357510,四川长虹


这次得到的结果仍然是垃圾股遍地

光我知道的就有<b>ST宏图、长虹、海航科技</b>等明星垃圾股

另外榜单几乎被基建行业的公司占领，这可能有以下几个原因：
- 基建行业目前确实非常低估，几个中字头的大公司PE都在3-4倍，市值低了指标当然会低
- 基建行业的经营与房地产有相似之处，都比较依赖大规模举债，所以账上现金多

到这一步其实不太好区分了，因为有些公司确实可能是错杀，有投资价值

但更多的是行业因素，或者企业有严重的财务造假嫌疑，不太具备投资价值

下面来看一下<b>交易性金融资产</b>的影响，交易性金融资产可以当作现金，但其波动巨大

另外一般有巨额交易性金融资产的都被认为是不务正业的公司，所以剔除这一项看看

In [12]:
# 剔除交易性资产，剔除特定行业
df = get_top_cashr(tra_ass=False, ex_fin=True)
df

,code,market_cap,cash_equivalents,trading_assets,cashr,display_name
2408,600153.XSHG,217.6300,5.840044e+10,1.182677e+09,2.683474,建发股份
2420,600170.XSHG,241.3092,6.456994e+10,2.240229e+09,2.675817,上海建工
2657,600502.XSHG,65.9204,1.351372e+10,7.000000e+07,2.050005,安徽建工
2921,600853.XSHG,24.1176,3.907026e+09,8.402000e+07,1.619990,龙建股份
2549,600339.XSHG,154.0949,2.282374e+10,NaN,1.481148,中油工程
3049,601186.XSHG,989.9486,1.375106e+11,2.579028e+09,1.389068,中国铁建
3104,601669.XSHG,610.4315,8.221579e+10,4.470244e+07,1.346847,中国电建
3103,601668.XSHG,1958.9794,2.623417e+11,2.454120e+08,1.339175,中国建筑
1122,002769.XSHE,23.1831,3.078929e+09,3.967474e+06,1.328092,普路通
92,000521.XSHE,34.6806,4.408281e+09,1.028681e+09,1.271109,长虹美菱


表中出现了很多交易性货币资产为0的公司，这几个公司的整体质量比之前的确实有所提升

但是我用其它的股票评分软件看了一下，大部分还是很垃圾

## 总结
到这里几乎就不用看了，这个指标十分弱鸡
- 指标行业聚集太强，可能前200都是金融、房地产、基建，如果一直剔除下去，也没太大意义了
- 筛选出不少垃圾股，这些公司有的已经陷入危机，有的是众所周知要陷入危机的，财务造假影响很大
- 指标的整体情况与前期预想的差异过大

<b>总之，完全不具备优秀指标的条件</b>，直接放弃了，没有回测的必要。这也提醒我拍脑门完全不可取。

